## 3.3.3 Realistic simulation setup
Different from the previous method, this method uses and ellipse estimation method and them estimates the centroid based on that.
In this script the goal is to remove all the graphs and obtain the results for a full series of chessboard for R and L sides

In [2]:
vscode = 1

In [3]:
if(vscode == 0):
    # for vscode
    # for jupyter notebook
    from mpl_toolkits.mplot3d import axes3d
    import matplotlib.pyplot as plt

    %matplotlib notebook
elif(vscode == 1):
    %matplotlib qt
    

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
 
from typing import Sequence
from calib_lib import *
from matplotlib.collections import PatchCollection      # pathcoletion for multiple patches
import matplotlib.cm as cm                              # for colormaps
from matplotlib.patches import Rectangle                # for rectangle representation
from matplotlib.patches import Ellipse                  # for ellipse representation   
from pixelization_lib import *      

DECIMALS = 2            # how many decimal places to use in print


In [5]:
import logging 
import numpy as np
from skimage.measure import EllipseModel
from matplotlib.patches import Ellipse
import matplotlib.pyplot as plt

In [6]:
F = 16                                       # focal length( in mm )
gridSize = 1024
subpixel_grid = 64
image_size = np.array([1936,1216])               # sensor size(in mm)
#image_size = np.array([11.345,7.126])
pixel_width = 5.86e-3                       # pixel size in mm
PX= image_size[0]/2.0                       # principal point_i x-coordinate
PY= image_size[1]/2.0                       # principal point_i y-coordinate
IMAGE_HEIGTH = image_size[1]
IMAGE_WIDTH = image_size[0]
THETA_X = 0                                 # roll angle
THETA_Y = 0                                 # pitch angle
THETA_Z = 0                                 # yaw angle

# camera Right
THETA_X_R = 0                                 # roll angle
THETA_Y_R = 0                                 # pitch angle
THETA_Z_R= 0                                 # yaw angle
# camera Left
THETA_X_L = 0                                 # roll angle
THETA_Y_L = 0                                 # pitch angle
THETA_Z_L= 0                                 # yaw angle

C_L = np.array([0,0,0])                     # camera centre
C_R = np.array([500,0,0])

chess_dimx,chess_dimy = (13,9)
chess_sq_size = 44

radius = 5

In [14]:
xrange_5 = np.linspace(200,400,6)
yrange_5 = np.linspace(-120,120,6)
zrange_5 = ([2000])

world_pts_arr,x_zhang_R,x_zhang_L,P_R,P_L = create_1024_point_chess(xrange_5,yrange_5,zrange_5)


np.save('world_pts_arr_180.npy',world_pts_arr)
np.save('x_zhang_R_180.npy',x_zhang_R)
np.save('x_zhang_L_180.npy',x_zhang_L)
 

(20, 140, 1024, 3)
(140, 1024, 4)
(20, 140, 1024, 3)


KeyboardInterrupt: 

In [ ]:
world_pts_arr = np.load('world_pts_arr_180.npy')
x_zhang_R = np.load('x_zhang_R_180.npy')
x_zhang_L = np.load('x_zhang_L_180.npy')

In [13]:
fig = plt.figure(figsize=(20,11))
fig.suptitle('Left and Right image coordinates', fontsize=16)
ax_L = fig.add_subplot(121)
for i in range(x_zhang_L.shape[0]):
    ax_L.plot(x_zhang_L[i,:,:,0],x_zhang_L[i,:,:,1], color = 'r',ls = "None", marker = ".", label = 'Original')
ax_L.set_xlabel("X axis")
ax_L.set_ylabel("Y axis")
ax_L.grid()
ax_L.set_xlim([0,image_size[0]])
ax_L.set_ylim([0,image_size[1]])


ax_R = fig.add_subplot(122)
for i in range(x_zhang_R.shape[0]):
    ax_R.plot(x_zhang_R[i,:,:,0],x_zhang_R[i,:,:,1], color = 'r',ls = "None", marker = ".", label = 'Original')
ax_R.set_xlabel("X axis")
ax_R.set_ylabel("Y axis")
ax_R.grid()
ax_R.set_xlim([0,image_size[0]])
ax_R.set_ylim([0,image_size[1]])
plt.show()

In [ ]:
x_zhang_R

In [ ]:
def spot_center_coord(cx0,cy0,pixelGridX,pixelGridY):
    cx0_max = np.where((cx0 < pixelGridX))
    cx0_min = np.where((cx0 > pixelGridX))
    #print(cx0_max[0][0])
    #print(cx0_min[0][-1])
    cy0_max = np.where((cy0 < pixelGridY))
    cy0_min = np.where((cy0 > pixelGridY))
    #print(cy0_max[0][0])
    #print(cy0_min[0][-1])

    return cx0_max[0][0],cx0_min[0][-1],cy0_max[0][0],cy0_min[0][-1]


def findcircle(x_arr):
    """
    Based on the image coordinates, and knowing it forms a square/rectangle, finds the centroid and radius

    Parameters
    ----------
    x_arr:  np.array
        image coordinates
    Returns
    -------
    cx0:    np.float
        centroid x
    cy0:    np.float
        centroid y    
    r0 : np.float
        circle radius
    """  
    
    Ax  = x_arr[0,0]
    Ay  = x_arr[0,1]
    Bx  = x_arr[int(x_arr.shape[0]/4),0]
    By  = x_arr[int(x_arr.shape[0]/4),1]
    Cx  = x_arr[int(x_arr.shape[0]/2),0]
    Cy  = x_arr[int(x_arr.shape[0]/2),1]
    Dx  = x_arr[int((x_arr.shape[0]/4*3)-1),0]
    Dy  = x_arr[int((x_arr.shape[0]/4*3)-1),1]


    cx0 = np.sum(x_arr[:,0])/np.sum(x_arr[:,0].shape[0])
    cy0 = np.sum(x_arr[:,1])/np.sum(x_arr[:,1].shape[0])

    r0 = np.abs((Dy-By)/2)
    
    return cx0,cy0,r0

def createcircle(x_arr,nrpoints = 100):
    """
    create the circle, using the function findcircle()

    Parameters
    ----------
    x_arr:  np.array
        image coordinates
    nrpoints: integer
        number of points that compose the circle
    Returns
    -------
    circle_x:    np.array
        points that define the circle in the cx0 coordinate
    circle_y:    np.array
        points that define the circle in the cy0 coordinate
    cx0:    np.float
        centroid x
    cy0:    np.float
        centroid y    
    r0 : np.float
        circle radius    
    """    

    cx0,cy0,r0 = findcircle(x_arr)

    theta = np.linspace(0, 2*np.pi, nrpoints)
    x = r0*np.cos(theta)
    y = r0*np.sin(theta)

    circle_x = x+cx0
    circle_y = y+cy0


    return circle_x, circle_y,cx0,cy0,r0

In [ ]:
x0_L = x_zhang_L[:,:,:,0]*pixel_width
y0_L = x_zhang_L[:,:,:,1]*pixel_width

x0_R = x_zhang_R[:,:,:,0]*pixel_width
y0_R = x_zhang_R[:,:,:,1]*pixel_width

### Find the center of the circle(point)

In [ ]:
# Generate pixel grid
pixelGridX  = (np.arange(0,image_size[0])) * pixel_width
pixelGridY  = (np.arange(0,image_size[1])) * pixel_width
pixelGridXX, pixelGridYY = np.meshgrid(pixelGridX,pixelGridY)
# magnification 
zoom_1 = 5
zoom_2 = 3
### Left image

#fig = plt.figure(figsize=(20,20))
# sup title
#fig.suptitle("Left and Right image plane coordinates")

#ax = fig.add_subplot()
#ax.plot(pixelGridXX,pixelGridYY,'b.-', label = 'Original')
#ax.plot(pixelGridXX.T,pixelGridYY.T, 'r.-', label = 'Original')


cx0_L = np.zeros((x_zhang_L.shape[0],x_zhang_L.shape[1],1))
cy0_L = np.zeros((x_zhang_L.shape[0],x_zhang_L.shape[1],1))
r0_L = np.zeros((x_zhang_L.shape[0],x_zhang_L.shape[1],1))
cx0_R = np.zeros((x_zhang_R.shape[0],x_zhang_R.shape[1],1))
cy0_R = np.zeros((x_zhang_R.shape[0],x_zhang_R.shape[1],1))
r0_R = np.zeros((x_zhang_R.shape[0],x_zhang_R.shape[1],1))

#
x0_cL = np.zeros((x_zhang_L.shape[0],x_zhang_L.shape[1],gridSize))
y0_cL = np.zeros((x_zhang_L.shape[0],x_zhang_L.shape[1],gridSize))
x0_cR = np.zeros((x_zhang_R.shape[0],x_zhang_R.shape[1],gridSize))
y0_cR = np.zeros((x_zhang_R.shape[0],x_zhang_R.shape[1],gridSize))


for chess_i in range(x0_L.shape[0]):
    for point_i in range(x0_L.shape[1]):

        #circle definition 
        # multiply by pixel_width to get the results in mm, not in pixels
        x0_cL[chess_i,point_i,:],y0_cL[chess_i,point_i,:],cx0_L[chess_i,point_i,:],cy0_L[chess_i,point_i,:],r0_L[chess_i,point_i,:] = createcircle(x_zhang_L[chess_i,point_i,:]*pixel_width,gridSize)
        x0_cR[chess_i,point_i,:],y0_cR[chess_i,point_i,:],cx0_R[chess_i,point_i,:],cy0_R[chess_i,point_i,:],r0_R[chess_i,point_i,:] = createcircle(x_zhang_R[chess_i,point_i,:]*pixel_width,gridSize)

      


### Find the center and axes of the ellipse( point )

In [ ]:
xc_arr_L = np.zeros((x0_L.shape[0],x0_L.shape[1],1))
yc_arr_L = np.zeros((x0_L.shape[0],x0_L.shape[1],1))
a_arr_L = np.zeros((x0_L.shape[0],x0_L.shape[1],1))
b_arr_L = np.zeros((x0_L.shape[0],x0_L.shape[1],1))
theta_arr_L = np.zeros((x0_L.shape[0],x0_L.shape[1],1))

xc_arr_R = np.zeros((x0_R.shape[0],x0_R.shape[1],1))
yc_arr_R = np.zeros((x0_R.shape[0],x0_R.shape[1],1))
a_arr_R = np.zeros((x0_R.shape[0],x0_R.shape[1],1))
b_arr_R = np.zeros((x0_R.shape[0],x0_R.shape[1],1))
theta_arr_R = np.zeros((x0_R.shape[0],x0_R.shape[1],1))

ell_L = EllipseModel()
ell_R = EllipseModel()


for chess_i in range(x0_L.shape[0]):
    for point_i in range(x0_L.shape[1]):


        X_L = (x_zhang_L[chess_i,point_i,:,:]*pixel_width).tolist()
        X_R = (x_zhang_R[chess_i,point_i,:,:]*pixel_width).tolist()


        pts_L = np.array(X_L)
        pts_R = np.array(X_R)

        ell_L.estimate(pts_L)
        ell_R.estimate(pts_R)


        xc_arr_L[chess_i,point_i,:], yc_arr_L[chess_i,point_i,:], a_arr_L[chess_i,point_i,:], b_arr_L[chess_i,point_i,:], theta_arr_L[chess_i,point_i,:] = ell_L.params
        xc_arr_R[chess_i,point_i,:], yc_arr_R[chess_i,point_i,:], a_arr_R[chess_i,point_i,:], b_arr_R[chess_i,point_i,:], theta_arr_R[chess_i,point_i,:] = ell_R.params

In [ ]:
xc_arr_R

### Find max and mim values that create boundaries in the ellipse and plot the center of the ellipse 
Also, meshgrid is added to the ellipse, representing the pixels

In [ ]:
x_L_lst = []
y_L_lst = []
for chess_b_i in range(x0_L.shape[0]):
    x_L_pts = []
    y_L_pts = []
    for point in range(x0_L.shape[1]):
        #ax.plot(x0_L[chess_b_i,point,:],y0_L[chess_b_i,point,:])
        max_pt_x_L = np.max(x_zhang_L[chess_b_i,point,:,0]*pixel_width)
        max_pt_y_L = np.max(x_zhang_L[chess_b_i,point,:,1]*pixel_width)
        min_pt_x_L = np.min(x_zhang_L[chess_b_i,point,:,0]*pixel_width)
        min_pt_y_L = np.min(x_zhang_L[chess_b_i,point,:,1]*pixel_width)

        pts = np.array([[max_pt_x_L,max_pt_y_L],[max_pt_x_L,min_pt_y_L],[min_pt_x_L,min_pt_y_L],[min_pt_x_L,max_pt_y_L]])
        cmin_pxl_L = np.zeros((pts.shape[0],pts.shape[1]))
        for pts_i,pts_d in enumerate(pts):
            cx0max_L,cx0min_L,cy0max_L,cy0min_L  = spot_center_coord(pts_d[0],pts_d[1],pixelGridX,pixelGridY)

            # max and min points in (x,y) coordinates
            cmin_pxl_L[pts_i,0] = (cx0min_L*pixel_width)
            cmin_pxl_L[pts_i,1] = (cy0min_L*pixel_width)

        x_L_pts.append(np.arange(cmin_pxl_L[3,0],cmin_pxl_L[0,0]+pixel_width,pixel_width))
        y_L_pts.append(np.arange(cmin_pxl_L[1,1],cmin_pxl_L[0,1]+pixel_width,pixel_width))
        xx,yy = np.meshgrid(x_L_pts[point],y_L_pts[point])

        #ax.plot(xx,yy, color = 'g',ls = "None", marker = "*")


    x_L_lst.append(x_L_pts)
    y_L_lst.append(y_L_pts)
#ax.set_aspect(1)    
#plt.show()

In [ ]:
x_R_lst = []
y_R_lst = []
for chess_b_i in range(x0_R.shape[0]):
    x_R_pts = []
    y_R_pts = []
    for point in range(x0_R.shape[1]):
        #ax.plot(x0_L[chess_b_i,point,:],y0_L[chess_b_i,point,:])
        max_pt_x_R = np.max(x_zhang_R[chess_b_i,point,:,0]*pixel_width)
        max_pt_y_R = np.max(x_zhang_R[chess_b_i,point,:,1]*pixel_width)
        min_pt_x_R = np.min(x_zhang_R[chess_b_i,point,:,0]*pixel_width)
        min_pt_y_R = np.min(x_zhang_R[chess_b_i,point,:,1]*pixel_width)

        pts = np.array([[max_pt_x_R,max_pt_y_R],[max_pt_x_R,min_pt_y_R],[min_pt_x_R,min_pt_y_R],[min_pt_x_R,max_pt_y_R]])
        cmin_pxl_R = np.zeros((pts.shape[0],pts.shape[1]))
        for pts_i,pts_d in enumerate(pts):
            cx0max_R,cx0min_R,cy0max_R,cy0min_R  = spot_center_coord(pts_d[0],pts_d[1],pixelGridX,pixelGridY)

            # max and min points in (x,y) coordinates
            cmin_pxl_R[pts_i,0] = (cx0min_R*pixel_width)
            cmin_pxl_R[pts_i,1] = (cy0min_R*pixel_width)

        x_R_pts.append(np.arange(cmin_pxl_R[3,0],cmin_pxl_R[0,0]+pixel_width,pixel_width))
        y_R_pts.append(np.arange(cmin_pxl_R[1,1],cmin_pxl_R[0,1]+pixel_width,pixel_width))
        xx,yy = np.meshgrid(x_R_pts[point],y_R_pts[point])

        #ax.plot(xx,yy, color = 'g',ls = "None", marker = "*")


    x_R_lst.append(x_R_pts)
    y_R_lst.append(y_R_pts)
#ax.set_aspect(1)    
#plt.show()

In [ ]:
from matplotlib.path import Path


xgrid = np.linspace(0,1,subpixel_grid)*pixel_width
ygrid = np.linspace(0,1,subpixel_grid)*pixel_width

chess_pts_L = []
chess_ptchs_L = []


centroid_L_arr = np.zeros((x0_L.shape[0],x0_L.shape[1],2))
# runs the chessboard
for chess in range(x0_L.shape[0]):

    lst_pts_L = []
    ptchs_pts_L = []
    # runs each point in the chessboard
    for point in range(x0_L.shape[1]):

        lstL = []
        ptchsL = []
        
        # meshgrid for the pixel
        [X_crns_L,Y_crns_L] = np.meshgrid(x_L_lst[chess][point],y_L_lst[chess][point])

        # for each pixel
        for i in range(X_crns_L.flatten().shape[0]):
            cx0max = X_crns_L.flatten()[i]
            cy0max = Y_crns_L.flatten()[i]

            # translate the left bottom corner
            x_pixel_L = cx0max+xgrid
            y_pixel_L = cy0max+ygrid
            # create sub-pixel grid
            xx_pixel_L, yy_pixel_L = np.meshgrid(x_pixel_L,y_pixel_L)

            x_L, y_L = xx_pixel_L.flatten(), yy_pixel_L.flatten()
            
            # if the first and 5th chessboard , then enter in the loop( because this are the ones with circles)
            if chess % 5 == 0:
                circ_L = np.sqrt((xx_pixel_L-cx0_L[chess,point,:])**2 + (yy_pixel_L-cy0_L[chess,point,:])**2) <= r0_L[chess,point,:]
                circ_corners_L = np.where(circ_L == True)
                xxgrid = np.array(circ_corners_L).T

            else:
                # calculate wheter a point is inside the ellipse or not
                cos_angle = np.cos(np.radians(180.-theta_arr_L[chess,point,0]*180/np.pi))
                sin_angle = np.sin(np.radians(180.-theta_arr_L[chess,point,0]*180/np.pi))

                x_c_L = x_L - xc_arr_L[chess,point,0]
                y_c_L = y_L - yc_arr_L[chess,point,0]

                xct_L = x_c_L * cos_angle - y_c_L * sin_angle
                yct_L = x_c_L * sin_angle + y_c_L * cos_angle 

                rad_cc = (xct_L**2/(a_arr_L[chess,point,0])**2) + (yct_L**2/(b_arr_L[chess,point,0])**2)

                # set the 
                xxgrid_L = np.where(rad_cc <= 1.)[0]

            lstL.append((xxgrid_L.shape[0]/(subpixel_grid**2))*100)

            ptchsL.append(Rectangle((cx0max,cy0max), pixel_width,pixel_width))


        lst_pts_L.append(lstL)
        ptchs_pts_L.append(ptchsL)
    
    chess_pts_L.append(lst_pts_L)
    chess_ptchs_L.append(ptchs_pts_L)

In [ ]:
from matplotlib.path import Path


xgrid = np.linspace(0,1,subpixel_grid)*pixel_width
ygrid = np.linspace(0,1,subpixel_grid)*pixel_width

chess_pts_R = []
chess_ptchs_R = []


centroid_R_arr = np.zeros((x0_R.shape[0],x0_R.shape[1],2))
# runs the chessboard
for chess in range(x0_R.shape[0]):

    lst_pts_R = []
    ptchs_pts_R = []
    # runs each point in the chessboard
    for point in range(x0_R.shape[1]):

        lstR = []
        ptchsR = []
        
        # meshgrid for the pixel
        [X_crns_R,Y_crns_R] = np.meshgrid(x_R_lst[chess][point],y_R_lst[chess][point])

        # for each pixel
        for i in range(X_crns_R.flatten().shape[0]):
            cx0max = X_crns_R.flatten()[i]
            cy0max = Y_crns_R.flatten()[i]

            # translate the left bottom corner
            x_pixel_R = cx0max+xgrid
            y_pixel_R = cy0max+ygrid
            # create sub-pixel grid
            xx_pixel_R, yy_pixel_R = np.meshgrid(x_pixel_R,y_pixel_R)

            x_R, y_R = xx_pixel_R.flatten(), yy_pixel_R.flatten()
            
            # if the first and 5th chessboard , then enter in the loop( because this are the ones with circles)
            if chess % 5 == 0:
                circ_R = np.sqrt((xx_pixel_R-cx0_R[chess,point,:])**2 + (yy_pixel_R-cy0_R[chess,point,:])**2) <= r0_R[chess,point,:]
                circ_corners_R = np.where(circ_R == True)
                xxgrid_R = np.array(circ_corners_R).T

            else:
                # calculate wheter a point is inside the ellipse or not
                cos_angle = np.cos(np.radians(180.-theta_arr_R[chess,point,0]*180/np.pi))
                sin_angle = np.sin(np.radians(180.-theta_arr_R[chess,point,0]*180/np.pi))

                x_c_R = x_R - xc_arr_R[chess,point,0]
                y_c_R = y_R - yc_arr_R[chess,point,0]

                xct_R = x_c_R * cos_angle - y_c_R * sin_angle
                yct_R = x_c_R * sin_angle + y_c_R * cos_angle 

                rad_cc = (xct_R**2/(a_arr_R[chess,point,0])**2) + (yct_R**2/(b_arr_R[chess,point,0])**2)

                # set the 
                xxgrid_R = np.where(rad_cc <= 1.)[0]

            lstR.append((xxgrid_R.shape[0]/(subpixel_grid**2))*100)

            ptchsR.append(Rectangle((cx0max,cy0max), pixel_width,pixel_width))


        lst_pts_R.append(lstR)
        ptchs_pts_R.append(ptchsR)
    
    chess_pts_R.append(lst_pts_R)
    chess_ptchs_R.append(ptchs_pts_R)

In [ ]:
centroid_L_arr = np.zeros((x0_L.shape[0],x0_L.shape[1],2))
for chess in range(x0_L.shape[0]):

    # runs each point in the chessboard
    for point in range(x0_L.shape[1]):

        [X_crns_L,Y_crns_L] = np.meshgrid(x_L_lst[chess][point],y_L_lst[chess][point])
        # to array transformation
        chess_pts_arr_L = np.array(chess_pts_L[chess][point])
    
        ##### centroid estimation ######
        centroid_L_x = np.sum(np.multiply(X_crns_L.flatten(),chess_pts_arr_L))/(np.sum(chess_pts_arr_L)) + pixel_width/2
        centroid_L_y = np.sum(np.multiply(Y_crns_L.flatten(),chess_pts_arr_L))/(np.sum(chess_pts_arr_L)) + pixel_width/2
        centroid_L_arr[chess,point,:]  = (centroid_L_x,centroid_L_y)


In [ ]:
centroid_R_arr = np.zeros((x0_R.shape[0],x0_R.shape[1],2))
for chess in range(x0_R.shape[0]):

    # runs each point in the chessboard
    for point in range(x0_R.shape[1]):

        [X_crns_R,Y_crns_R] = np.meshgrid(x_R_lst[chess][point],y_R_lst[chess][point])
        # to array transformation
        chess_pts_arr_R = np.array(chess_pts_R[chess][point])

        ##### centroid estimation ######
        centroid_R_x = np.sum(np.multiply(X_crns_R.flatten(),chess_pts_arr_R))/(np.sum(chess_pts_arr_R)) + pixel_width/2
        centroid_R_y = np.sum(np.multiply(Y_crns_R.flatten(),chess_pts_arr_R))/(np.sum(chess_pts_arr_R)) + pixel_width/2
        centroid_R_arr[chess,point,:]  = (centroid_R_x,centroid_R_y)


**NOTES:** The script now works for either the circle and the ellipse. The next step is to remove the graphs and do for a complete series of chessboard for R and L sides.